In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
tf.test.is_built_with_cuda()
import torch
import torch.nn as nn 
import math
from torch import nn, Tensor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import pyarrow.parquet as pq

from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

import random
# Set the seed value all over the place to make this reproducible.
seed_val = 42


random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



In [2]:
class ExobootDataset(Dataset):
    """Windowed Gait dataset."""

    def __init__(self, gait_data, window_size=50, meas_scale=None, speed_scale=None, incline_scale=None, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            window_size (integer): size of the window to apply to the data
            speed_scale (tuple, optional): A 2-tuple of the lower and upper bounds to scale the speed
            incline_scale (tuple, optional): A 2-tuple of the lower and upper bounds to scale the incline
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.gait_data = gait_data
        self.window_size = window_size
        self.meas_scale = meas_scale
        self.speed_scale = speed_scale
        self.incline_scale = incline_scale
        self.transform = transform

    def __len__(self):
        return len(self.gait_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        if idx < self.window_size and idx >= 0:
          # idx = len(self.gait_data)-1
          idx = random.randint(self.window_size, len(self.gait_data)-1)
          # print('saturating')


        meas_idxs = [0,1,2,3,4,5,6]

        #phase, speed, incline, is_stairs, is_moving
        gait_state_idxs = [7,8,9,10,11]  
        measurements = self.gait_data.iloc[idx-self.window_size:idx,meas_idxs].to_numpy()
        
        gait_states = self.gait_data.iloc[idx,gait_state_idxs].to_numpy()

        # sample = {'meas': measurements, 'state': gait_states}
        
        if self.meas_scale is not None:
            for i in range(len(meas_idxs)):
                lb = self.meas_scale[i,0]
                ub = self.meas_scale[i,1]
                measurements[:,i] = ((1 - 0)/(ub - lb)) * (measurements[:,i] - lb)

        if self.speed_scale:
            lb = self.speed_scale[0]
            ub = self.speed_scale[1]
            gait_states[1] = ((1 - 0)/(ub - lb)) * (gait_states[1] - lb)

        if self.incline_scale:
            lb = self.incline_scale[0]
            ub = self.incline_scale[1]
            gait_states[2] = ((1 - 0)/(ub - lb)) * (gait_states[2] - lb)

        
        phase_as_angle = 2*np.pi*(gait_states[0]-0.5)
        cp = np.cos(phase_as_angle)
        sp = np.sin(phase_as_angle)
        gait_states_new = np.zeros(gait_states.shape[0]+1,)
        gait_states_new[0] = cp
        gait_states_new[1] = sp
        gait_states_new[2:] = gait_states[1:]
        gait_states = gait_states_new

        sample = {'meas': measurements, 'state': gait_states}

        if self.transform:
            sample = self.transform(sample, self.window_size)
        
        return sample
        
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample, window_size):
        meas, state = sample['meas'], sample['state']
       
        meas = torch.from_numpy(meas).float()
        state = torch.from_numpy(state).float()
        state = torch.unsqueeze(state, dim=0)

        return {'meas': meas, 
                'state': state}



In [3]:
#RUN THIS TO SET UP COLAB
ON_COLAB = True
if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/Phase ML Data/'

Mounted at /content/drive


In [5]:
window_size = 100
meas_scale = np.array([[-69.35951035,  27.62815047],\
                            [-456.18013759,  401.13782617],\
                            [-63.71649984,  22.06632622],\
                            [-213.4786175,   396.93801619],\
                            [-35.26603985,  20.78473636],\
                            [-20.95456523,  14.63961137],\
                              [0,1]])
speed_scale = (0,2)
incline_scale = (-10,10)

filename = 'exoboot_mars6.csv'

if ON_COLAB:
  filename = drive_path + filename

exoboot_data = pd.read_csv(filename)

print(exoboot_data.head() )
print(len(exoboot_data))




   foot_angles  foot_vel_angles  shank_angles  shank_vel_angles  \
0    27.703737       -22.952617     21.832302        -63.502207   
1    25.742622       -65.596793     21.297953        -86.185133   
2    24.324775      -100.132783     20.534140       -118.166841   
3    24.324775      -119.138618     19.465094       -134.508304   
4    23.294041       -35.650328     18.226569        -68.380255   

   heel_acc_forward  heel_acc_up      dt  phase_ground_truth  \
0        -24.857099    -5.109614  0.0100            0.008016   
1        -21.149576    -3.931122  0.0099            0.983308   
2        -17.943035    -3.000270  0.0100            0.983729   
3        -17.769398    -2.557593  0.0100            0.989832   
4         13.277327    18.289915  0.0100            0.994453   

   speed_ground_truth  incline_ground_truth  is_stairs  is_moving  
0            0.670840             -1.419898        0.0        1.0  
1            0.669866             -1.418597        0.0        1.0  
2       

In [6]:
class PositionalEncoder(nn.Module):
    """
    The authors of the original transformer paper describe very succinctly what 
    the positional encoding layer does and why it is needed:
    
    "Since our model contains no recurrence and no convolution, in order for the 
    model to make use of the order of the sequence, we must inject some 
    information about the relative or absolute position of the tokens in the 
    sequence." (Vaswani et al, 2017)
    Adapted from: 
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(
        self, 
        dropout: float=0.1, 
        max_seq_len: int=100, 
        d_model: int=512,
        batch_first: bool=True,
        ):

        """
        Parameters:
            dropout: the dropout rate
            max_seq_len: the maximum length of the input sequences
            d_model: The dimension of the output of sub-layers in the model 
                     (Vaswani et al, 2017)
        """

        super().__init__()

        self.d_model = d_model
        
        self.dropout = nn.Dropout(p=dropout)

        self.batch_first = batch_first
        self.max_seq_len = max_seq_len
        self.x_dim = 1 if batch_first else 0
        pe = torch.zeros((1,max_seq_len,d_model))
        self.register_buffer('pe', pe, persistent=False)

        
    def forward(self, x: Tensor, dts: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [batch_size, enc_seq_len, dim_val] or 
               [enc_seq_len, batch_size, dim_val]
        """
        
        #dts should be (batch_size, 50, 1)
        #print('2')
        #print(dts.shape)
  
        t_rel = torch.cumsum(torch.flip(dts, [1]), dim=1)

        #flip order so most recent integrated is last
        t_rel = torch.flip(t_rel, [1])
        
        #repeat integrated time across the embedding dimension
        t_rel = t_rel.repeat(1, 1, self.d_model)

        self.pe[0,:-1,:] = t_rel[0,1:,:]
        
        # self.pe = self.pe.to(device)
        x = x + self.pe

        return self.dropout(x)

class GaitTransformer(nn.Module):

    """
    This class implements a transformer model that can be used for times series
    forecasting. This time series transformer model is based on the paper by
    Wu et al (2020) [1]. The paper will be referred to as "the paper".
    A detailed description of the code can be found in my article here:
    https://towardsdatascience.com/how-to-make-a-pytorch-transformer-for-time-series-forecasting-69e073d4061e
    In cases where the paper does not specify what value was used for a specific
    configuration/hyperparameter, this class uses the values from Vaswani et al
    (2017) [2] or from PyTorch source code.
    Unlike the paper, this class assumes that input layers, positional encoding 
    layers and linear mapping layers are separate from the encoder and decoder, 
    i.e. the encoder and decoder only do what is depicted as their sub-layers 
    in the paper. For practical purposes, this assumption does not make a 
    difference - it merely means that the linear and positional encoding layers
    are implemented inside the present class and not inside the 
    Encoder() and Decoder() classes.
    [1] Wu, N., Green, B., Ben, X., O'banion, S. (2020). 
    'Deep Transformer Models for Time Series Forecasting: 
    The Influenza Prevalence Case'. 
    arXiv:2001.08317 [cs, stat] [Preprint]. 
    Available at: http://arxiv.org/abs/2001.08317 (Accessed: 9 March 2022).
    [2] Vaswani, A. et al. (2017) 
    'Attention Is All You Need'.
    arXiv:1706.03762 [cs] [Preprint]. 
    Available at: http://arxiv.org/abs/1706.03762 (Accessed: 9 March 2022).
    """

    def __init__(self, 
        input_size: int,
        num_predicted_features: int=4,
        batch_first: bool=True,
        dim_val: int=512,  
        n_encoder_layers: int=4,
        n_decoder_layers: int=4,
        n_heads: int=8,
        enc_seq_len: int=50,
        dropout_encoder: float=0, 
        dropout_decoder: float=0,
        dropout_pos_enc: float=0,
        dropout_regression: float = 0,
        dim_feedforward_encoder: int=128,
        dim_feedforward_decoder: int=128,
        ): 

        """
        Args:
            input_size: int, number of input variables. 1 if univariate.
            dec_seq_len: int, the length of the input sequence fed to the decoder
            dim_val: int, aka d_model. All sub-layers in the model produce 
                     outputs of dimension dim_val
            n_encoder_layers: int, number of stacked encoder layers in the encoder
            n_heads: int, the number of attention heads (aka parallel attention layers)
            dropout_encoder: float, the dropout rate of the encoder
            dropout_pos_enc: float, the dropout rate of the positional encoder
            dim_feedforward_encoder: int, number of neurons in the linear layer 
                                     of the encoder
            num_predicted_features: int, the number of features you want to predict.
                                    Most of the time, this will be 1 because we're
                                    only forecasting FCR-N prices in DK2, but in
                                    we wanted to also predict FCR-D with the same
                                    model, num_predicted_features should be 2.
        """

        super().__init__() 

        #print("input_size is: {}".format(input_size))
        #print("dim_val is: {}".format(dim_val))

        # Creating the three linear layers needed for the model
        self.embedding_layer = nn.Linear(
            in_features=input_size, 
            out_features=dim_val 
            )


        # The encoder layer used in the paper is identical to the one used by
        # Vaswani et al (2017) on which the PyTorch module is based.
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim_val, 
            nhead=n_heads,
            dim_feedforward=dim_feedforward_encoder,
            dropout=dropout_encoder,
            activation='gelu',
            batch_first=batch_first
            )
        
        self.decoder_embedding_layer = nn.Linear(
            in_features=num_predicted_features,
            out_features=dim_val
            )
        
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=dim_val,
            nhead=n_heads,
            dim_feedforward=dim_feedforward_decoder,
            dropout=dropout_decoder,
            activation='gelu',
            batch_first=batch_first
            )

        # Create positional encoder
        self.positional_encoding_layer = PositionalEncoder(
            d_model=dim_val,
            dropout=dropout_pos_enc,
            max_seq_len=enc_seq_len
            )
        
        # Stack the encoder layers in nn.TransformerEncoder
        # It seems the option of passing a normalization instance is redundant
        # in my case, because nn.TransformerEncoderLayer per default normalizes
        # after each sub-layer
        # (https://github.com/pytorch/pytorch/issues/24930).
        self.encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layer,
            num_layers=n_encoder_layers, 
            norm=None
            )
        
        # Stack the decoder layers in nn.TransformerDecoder
        # It seems the option of passing a normalization instance is redundant
        # in my case, because nn.TransformerDecoderLayer per default normalizes
        # after each sub-layer
        # (https://github.com/pytorch/pytorch/issues/24930).
        self.decoder = nn.TransformerDecoder(
            decoder_layer=decoder_layer,
            num_layers=n_decoder_layers, 
            norm=None
            )

        # Regression head
     
        self.regression_head = nn.Sequential(
            nn.LayerNorm(dim_val),
            nn.Dropout(dropout_regression),
            nn.Linear(dim_val,  num_predicted_features)
        )


       
  

    def forward(self, src: Tensor, tgt: Tensor, dts : Tensor) -> Tensor:
        """
        Returns a tensor of shape:
        [target_sequence_length, batch_size, num_predicted_features]
        
        Args:
            src: the encoder's output sequence. Shape: (S,E) for unbatched input, 
                 (S, N, E) if batch_first=False or (N, S, E) if 
                 batch_first=True, where S is the source sequence length, 
                 N is the batch size, and E is the number of features (1 if univariate)
           
           tgt: the sequence to the decoder. Shape: (T,E) for unbatched input, 
                 (T, N, E)(T,N,E) if batch_first=False or (N, T, E) if 
                 batch_first=True, where T is the target sequence length, 
                 N is the batch size, and E is the number of features (1 if univariate)
            
        """

        #print("From model.forward(): Size of src as given to forward(): {}".format(src.size()))

        # Pass throguh the input layer right before the encoder
        src = self.embedding_layer(src) # src shape: [batch_size, src length, dim_val] regardless of number of input features
        #print("From model.forward(): Size of src after input layer: {}".format(src.size()))

        src = self.positional_encoding_layer(src, dts) # src shape: [batch_size, src length, dim_val] regardless of number of input features

        ## ENCODER
      
        src = self.encoder( # src shape: [batch_size, enc_seq_len, dim_val]
            src=src
            )
       
        decoder_output = self.decoder_embedding_layer(tgt) # tgt shape: [batch_size, target sequence length, dim_val] regardless of number of input features
        
        # Pass throguh decoder - output shape: [batch_size, target seq len, dim_val]
        decoder_output = self.decoder(
            tgt=decoder_output,
            memory=src
            )
        
        
        
        # Pass through regression mapping
        output = self.regression_head(decoder_output)
        
        # force the outputs of is_stairs and is_moving to be in [0,1] via sigmoid
        output[:,:,4] = torch.sigmoid(output[:,:,4])
        output[:,:,5] = torch.sigmoid(output[:,:,5])

        return output



In [8]:
#Test model
# Create the DataLoader.

test_dataset = ExobootDataset(gait_data=exoboot_data,
                                meas_scale=meas_scale,
                                window_size = window_size,
                                speed_scale = speed_scale,
                                incline_scale = incline_scale,
                                transform=ToTensor())

BATCH_SIZE = 1024*4
prediction_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=False,num_workers=8)
# prediction_dataloader = validation_dataloader
# Prediction on test set

print('Predicting labels for {:,} test points...'.format(len(test_dataset)))

# Model parameters
dim_val = 64 # This can be any value divisible by n_heads. 512 is used in the original transformer paper.
n_heads = 8 # The number of attention heads (aka parallel attention layers). dim_val must be divisible by this number
n_encoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
n_decoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
input_size = 7 # The number of input variables. 1 if univariate forecasting.
enc_seq_len = 100 # length of input given to encoder. Can have any integer value.
dec_seq_len = 1 # length of input given to decoder. Can have any integer value.

dropout_encoder = 0.1
dropout_decoder = 0.1
dropout_pos_enc = 0.0
dropout_regression = 0.1 
dim_feedforward_encoder = 1024
dim_feedforward_decoder = 1024

num_predicted_features = 6 # The number of output variables. 

#INITIALIZE MODEL
model = GaitTransformer(
    dim_val=dim_val,
    input_size=input_size, 
    n_encoder_layers=n_encoder_layers,
    n_decoder_layers=n_decoder_layers,
    n_heads=n_heads,
    enc_seq_len=enc_seq_len,
    dropout_encoder=dropout_encoder,
    dropout_decoder=dropout_decoder,
    dropout_pos_enc=dropout_pos_enc,
    dropout_regression=dropout_regression,
    num_predicted_features=num_predicted_features,
    dim_feedforward_encoder=dim_feedforward_encoder,
    dim_feedforward_decoder=dim_feedforward_decoder,
)

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

model.to(device)
ON_COLAB = True
model_nickname = 'keter'

#LOAD IN CHECKPOINTS
if ON_COLAB:
  general_model_dir = f'{drive_path}/full_models/{model_nickname}/model_save_xval/'
else:
  general_model_dir = f'./full_models/{model_nickname}/model_save_xval/'

checkpoint = torch.load(general_model_dir+'ml_gait_estimator_dec_best_model.tar',map_location=torch.device(device))
g = checkpoint['model_state_dict']
loss = checkpoint['loss']
print(f'Lowest Loss General: {loss}')
model.load_state_dict(g)

# Put models in evaluation mode
model.eval()

# Tracking variables 
predictions = []
true_labels = []

#Set up start of sequence token
SOS_token = 100 * torch.ones(1, 1, num_predicted_features).to(device).requires_grad_(False)
#Extract index for time steps
DT_IDX = 6

def phase_dist(phase_a, phase_b):
    """computes a distance that accounts for the modular arithmetic of phase
    and guarantees that the output is between 0 and .5
    
    Args:
        phase_a (float): a phase between 0 and 1
        phase_b (float): a phase between 0 and 1
    
    Returns:
        dist_prime: the difference between the phases, modulo'd between 0 and 0.5
    """
    if isinstance(phase_a, np.ndarray):
        dist_prime = (phase_a-phase_b)
        dist_prime[dist_prime > 0.5] = 1-dist_prime[dist_prime > 0.5]

        dist_prime[dist_prime < -0.5] = -1-dist_prime[dist_prime < -0.5]

    else:
        dist_prime = (phase_a-phase_b)
        if dist_prime > 0.5:
            dist_prime = 1-dist_prime

        elif dist_prime < -0.5:
            dist_prime = -1-dist_prime
    return dist_prime


def unscale_gait_state(gait_state_vec, speed_scale, incline_scale):
    rows, cols = gait_state_vec.shape
    # print(gait_state_vec.shape)
    gait_state_unscaled = np.zeros((rows,cols-1))
    
    cp = gait_state_vec[:,0]
    sp = gait_state_vec[:,1]
    
    #undo the trig on phase
    x = np.arctan2(sp,cp)
    phase_p = ((x)/(2*np.pi)) + 0.5
    
    gait_state_unscaled[:,0] = phase_p
    
    #unscale speed
    speed_lb = speed_scale[0]
    speed_ub = speed_scale[1]
    speed_unscaled = (gait_state_vec[:,2] * (speed_ub - speed_lb)) + speed_lb
    gait_state_unscaled[:,1] = speed_unscaled
    
    #unscale incline
    incline_lb = incline_scale[0]
    incline_ub = incline_scale[1]
    incline_unscaled = (gait_state_vec[:,3] * (incline_ub - incline_lb)) + incline_lb
    gait_state_unscaled[:,2] = incline_unscaled
    
    #add rest of gait state variables
    gait_state_unscaled[:,3:] = gait_state_vec[:,4:]
    
    return gait_state_unscaled
    
# Predict 
print(len(prediction_dataloader))
for batch in prediction_dataloader:

    b_input = batch['meas'].to(device)
    b_state = batch['state'].to(device)
    
    #truncate booleans
    # b_state = b_state[:,:,:4]
        
    tgt = SOS_token.repeat(b_state.shape[0], 1, 1)
    dts = b_input[:,:,DT_IDX]
    dts = torch.unsqueeze(dts, dim=-1)


    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input,tgt, dts)


    # Move logits and labels to CPU
    outputs = outputs.detach().to('cpu').numpy()
    b_state = b_state.to('cpu').numpy()
  
    # Store predictions and true labels
    outputs = np.squeeze(outputs, axis=1)
    b_state = np.squeeze(b_state, axis=1)
    
    # print(outputs.shape)
    
    #unscale
    outputs = unscale_gait_state(outputs, speed_scale, incline_scale)
    b_state = unscale_gait_state(b_state, speed_scale, incline_scale)

    # Store predictions and true labels
    predictions.extend(outputs.tolist())
    true_labels.extend(b_state.tolist())

print('    DONE PREDICTING')



Predicting labels for 2,916 test points...
Using GPU.
Lowest Loss General: 0.16128962696268317
1


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


    DONE PREDICTING


In [ ]:
predictions = np.array(predictions)
true_labels = np.array(true_labels)

#generate losses for general model
phase_losses = np.sqrt(phase_dist(predictions[:,0], true_labels[:,0])**2)
speed_losses = np.sqrt((predictions[:,1] - true_labels[:,1])**2)
incline_losses = np.sqrt((predictions[:,2] - true_labels[:,2])**2)
is_stairs_accuracy = np.sum(np.round(true_labels[:,3]) == np.round(predictions[:,3]))/len(true_labels[:,3])
is_moving_accuracy = np.sum(np.round(true_labels[:,4]) == np.round(predictions[:,4]))/len(true_labels[:,4])
print(predictions.shape)


print("="*30)
print('General Model')
print(f'Phase Losses: {np.mean(phase_losses):.3f} +- {np.std(phase_losses):.3f}')
print(f'Speed Losses: {np.mean(speed_losses):.3f} +- {np.std(speed_losses):.3f}')
print(f'Incline Losses: {np.mean(incline_losses):.3f} +- {np.std(incline_losses):.3f}')

print(f'Is Stairs Accuracy: {is_stairs_accuracy:.3f}')
print(f'Is Moving Accuracy: {is_moving_accuracy:.3f}')



fig, axs = plt.subplots(5,1,figsize=(20,12),sharex=True)
axs[0].plot(predictions[:,0],'r',label='predict, general')
axs[0].plot(true_labels[:,0],'b',label='actual')
axs[0].legend()
axs[0].set_ylabel('Phase')
# axs[0].set_xlim([80,5000])


axs[1].plot(predictions[:,1],'r')
axs[1].plot(true_labels[:,1],'b',label='actual')
axs[1].set_ylabel('Speed (m/s)')
axs[1].set_ylim([0,2])


axs[2].plot(predictions[:,2],'r')
axs[2].plot(true_labels[:,2],'b',label='actual')
axs[2].set_ylabel('Incline (deg)')
# axs[2].set_ylim([-15,15])

axs[3].plot(predictions[:,3],'r')
axs[3].plot(true_labels[:,3],'b',label='actual')
axs[3].set_ylabel('Is Stairs')
axs[3].set_ylim([-0.2,1.2])

axs[4].plot(predictions[:,4],'r')
axs[4].plot(true_labels[:,4],'b',label='actual')
axs[4].set_ylabel('Is Moving')
axs[4].set_ylim([-0.2,1.2])


(2916, 5)
General Model
Phase Losses: 0.029 +- 0.026
Speed Losses: 0.150 +- 0.110
Incline Losses: 3.116 +- 3.104
Is Stairs Accuracy: 0.849
Is Moving Accuracy: 1.000


(-0.2, 1.2)

In [ ]:
params = list(model.named_parameters())
print('The model has {:} different named parameters.\n'.format(len(params)))
for p in params:
    # print('p')
    # print(p[0])
    # print(p[1].data)
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)
